# Model Inferencing Notebook

## Research Title: Utilizing AI and IoT for Climate Adaptation
### Subtitle: Forecasting, Directing, and Storing Extreme Precipitation Events for Flood Prevention and Water Security

### By Krishna Nidamarthi, Emerald High School, Dublin CA.

#### About this research:

#### Abstract

Climate change is intensifying extreme precipitation events, leading to increased flooding while alternatively creating longer drought periods that threaten water availability. This research presents an integrated approach to adapt to this climate challenge in the short term by combining climate modeling, artificial intelligence & machine learning (AIML), Internet of Things (IoT) and cloud technologies. The study developed a three-pronged solution: (1) high-resolution precipitation forecast analysis with downscaling, (2) AI-powered satellite image analysis to rapidly identify potential water storage areas, and (3) IoT-enabled dynamic water management infrastructure using remote monitoring and control via the cloud. Using Merced County, California as a test case, the research employed the MACA-CCSM4 downscaling datasets to forecast future precipitation patterns. The analysis revealed a 150% increase in extreme precipitation over Sierra Nevada for 2026-2050 and 250% for 2051-2099 compared to 2000-2024 which escalated flood risk for Merced County. A custom convolutional neural network using Resnet50 with Feature Pyramid Network (FPN) was developed to analyze satellite imagery to identify suitable water reservoir areas. The proposed solution includes engineered wetlands and methods for constructing them in the identified reservoir areas, incorporating cloud-connected IoT sensors and remotely operated controls. These wetlands are capable of storing up to 2.5x106 cubic meters of surface water. This comprehensive approach demonstrates the potential of technology-driven climate adaptation strategies to simultaneously address flood prevention and water security challenges in vulnerable regions.

#### About this Jupyter Notebook

This notebook is the accompanying ML model inference code and it used the ML model that I trained as part of my research. The model and sample images are available in my Github. The model is a resnet50 CNN trained model, with FPN, and used to classify Google maps satellite tiles (zoom level 12, 512x512 pixels resolution [at scale 2], and without cloud cover) for 1) Farms, 2) Lakes, 3) Reservoir-Areas, and 4) Suburbs. 

Libraries and initialization. The Python environment in Jupyter lab must be setup with the following libraries.
With regards to the compatibility I used: pytorch==2.3.1 torchvision==0.18.1 torchaudio==2.3.1 pytorch-cuda=11.8

In [444]:
import torch
import torchvision
from PIL import Image, ImageDraw
import xml.etree.ElementTree as ET
import os
import matplotlib.pyplot as plt
import numpy as np
from torchvision.transforms import functional as F
from torchvision.models import resnet50, ResNet50_Weights
from torchvision.models.detection.backbone_utils import BackboneWithFPN
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor
from torchvision.models.detection import MaskRCNN
from datetime import datetime
from typing import Tuple, List
import matplotlib.patches as patches
from tqdm import tqdm

I use CVAT 1.1 for annotations and the following are the classes (labels) identified in a given Google maps satellite tile.

In [ ]:
# CVAT label configuration
LABEL_CONFIG = {
    'Farm': '#d08e1f',
    'Lake': '#93e4ca',
    'Reservoir-Area': '#3c3ffc',
    'Suburb': '#ff355e'
}

# GLOBAL CONSTANTS
DEBUG_MODE = False # Set to False for normal execution
WARN_MODE = False # False to suppress warning printouts

Tensor class compoistion (same as training module)

In [ ]:
class Compose:
    def __init__(self, transforms):
        self.transforms = transforms

    def __call__(self, image, target):
        for t in self.transforms:
            image, target = t(image, target)
        return image, target

class ToTensor:
    def __call__(self, image, target):
        image = F.to_tensor(image)
        return image, target

class Normalize:
    def __init__(self, mean, std):
        self.mean = mean
        self.std = std

    def __call__(self, image, target):
        image = F.normalize(image, mean=self.mean, std=self.std)
        return image, target

BoxRCNN class from training (same class as defined in training notebook)

In [ ]:
class BoxRCNN(torchvision.models.detection.MaskRCNN):
    def __init__(self, num_classes, class_to_idx):
        
        # Load a pre-trained ResNet50 model
        backbone = resnet50(weights=ResNet50_Weights.DEFAULT)        
        # Use the first 4 stages of ResNet
        backbone = torch.nn.Sequential(*list(backbone.children())[:-1])
        # Define the return layers for FPN
        return_layers = {'4': '0', '5': '1', '6': '2', '7': '3'}
        
        # Create FPN
        backbone_with_fpn = BackboneWithFPN(
            backbone,
            return_layers,
            in_channels_list=[256, 512, 1024, 2048],
            out_channels=256
        )
        # Initialize the MaskRCNN with our custom backbone
        super(BoxRCNN, self).__init__(backbone_with_fpn, num_classes)

        # Store class mappings
        self.class_to_idx = class_to_idx
        self.idx_to_class = {v: k for k, v in class_to_idx.items()}
        
        # Replace the box predictor
        in_features = self.roi_heads.box_predictor.cls_score.in_features
        self.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)
        
        # Replace the mask predictor
        in_features_mask = self.roi_heads.mask_predictor.conv5_mask.in_channels
        hidden_layer = 256
        self.roi_heads.mask_predictor = MaskRCNNPredictor(in_features_mask, hidden_layer, num_classes)

        #self.idx_to_class[0] = 'background' # explicitly additional background class
        #background class is area in the image where there is no classified image. i.e, BACKGROUND.

    def forward(self, images, targets=None):
        # For training mode
        if self.training and targets is not None:
            # Keep original string labels for later comparison
            for t in targets:
                 # Check if labels exist and are not empty
                if 'labels' not in t or len(t['labels']) == 0:
                    if TRAIN_DEBUG: print("Warning: Empty labels in target")
                    # Initialize with empty tensors
                    t['labels'] = torch.zeros(0, dtype=torch.int64, device=images[0].device)
                    continue
                
                if len(t['labels']) > 0 and isinstance(t['labels'][0], str):
                    try:
                         t['labels'] = torch.tensor([self.class_to_idx[label] for label in t['labels']], 
                                                    dtype=torch.int64, device=images[0].device)
                    except KeyError as e:
                        print(f"Error: Unknown label encountertered: {e}")
                        print(f"Available classes: {self.class_to_idx.keys()}")
                        raise
                    except Exception as e:
                        print(f"Error converting labels: {str(e)}")
                        print(f"Labels: {t['labels']}")
                        raise
        
        # MaskRCNN forward pass (works with indices)
        outputs = super(BoxRCNN, self).forward(images, targets)

        # Convert back to strings for external use if not in training mode
        if not self.training:
            for output in outputs:
                if 'labels' not in output or len(output['labels']) == 0:
                    if DEBUG_MODE: print("No detections in MaskRCNN forward pass output")
                    output['labels'] = []
                    continue

                try:
                    output['labels'] = [self.idx_to_class[label.item()] 
                                        if isinstance(label, torch.Tensor)
                                        else self.idx_to_class[label]
                                        for label in output['labels']]
                except Exception as e:
                    print(f"Error converting MaskRCNN forward pass output labels: {str(e)}")
                    print(f"Output labels: {output['labels']}")
                    raise

        return outputs

    def detect(self, images):
        # Helper method for inference
        self.eval()
        with torch.no_grad():
            predictions = self(images)  # This calls the forward method

            if TRAIN_DEBUG: print('In BoxRCNN.detect: ')
            
            for pred in predictions:
                # Check if labels exist and are not empty
                if 'labels' not in pred or len(pred['labels']) == 0:
                    if TRAIN_DEBUG:
                        print("No detections in this image")
                    # Initialize with empty lists/tensors
                    pred['boxes'] = torch.empty((0, 4), device=pred['boxes'].device)
                    pred['labels'] = []
                    pred['scores'] = torch.empty(0, device=pred['scores'].device)
                    pred['masks'] = torch.empty((0, 1, pred['masks'].shape[2], pred['masks'].shape[3]), 
                                             device=pred['masks'].device)
                    continue
                
                # Convert labels to strings if they aren't already
                if not isinstance(pred['labels'][0], str):
                    pred['labels'] = [self.idx_to_class[label.item()] 
                                    if isinstance(label, torch.Tensor) 
                                    else self.idx_to_class[label] 
                                    for label in pred['labels']]
                
                if TRAIN_DEBUG:
                    print("In BoxRCNN.detect:")
                    print(f"Number of detections: {len(pred['labels'])}")
                    print(f"Labels: {pred['labels']}")
                    print(f"Scores: {pred['scores']}")
                    print(f"Label types: {[type(label) for label in pred['labels']]}")
        
        return predictions

Utilities

In [ ]:
# color for bounding boxes
def hex_to_rgb(hex_color: str) -> Tuple[int, int, int]:
    """Convert hex color to RGB tuple"""
    hex_color = hex_color.lstrip('#')
    return tuple(int(hex_color[i:i+2], 16) for i in (0, 2, 4))

Check CUDA available, and load ML Model to the appropriate device. CUDA is preferred if available.

In [ ]:
def get_device():
    if torch.cuda.is_available():
        # Check the number of available CUDA devices
        device_count = torch.cuda.device_count()
        if device_count > 0:
            # If multiple devices are available, try to find the NVIDIA GPU
            for i in range(device_count):
                device_name = torch.cuda.get_device_name(i)
                if 'NVIDIA' in device_name or 'RTX' in device_name:
                    print(f"Using CUDA device: {torch.cuda.get_device_name(i)}")
                    return torch.device(f'cuda:{i}')
            
            # If NVIDIA GPU not found, use the first available CUDA device
            print(f"No NVIDIA GPU, but using CUDA device: {torch.cuda.get_device_name(0)}")
            return torch.device('cuda:0')
    
    # If no CUDA devices are available, use CPU
    print("CUDA is not available. Using CPU.")
    return torch.device('cpu')

Load ML Model. 

In [ ]:
def load_checkpoint_model(checkpoint_path: str) -> torch.nn.Module:
    """
    Load model from checkpoint file
    Args:
        checkpoint_path: Path to the .pth checkpoint file
    Returns:
        Loaded model
    """
    device = get_device()
    
    # Load checkpoint
    checkpoint = torch.load(checkpoint_path, map_location=device, weights_only=True)
    
    # Initialize model with correct number of classes (4 classes + background)
    model = BoxRCNN(num_classes=5, class_to_idx={k: i+1 for i, k in enumerate(LABEL_CONFIG.keys())})
    
    # Load state dict
    if 'model_state_dict' in checkpoint:
        # This is a training checkpoipoint
        model.load_state_dict(checkpoint['model_state_dict'])
        print(f"Loaded checkpoint from epoch {checkpoint['epoch']}")
    else:
        # This is just the state dict
        model.load_state_dict(checkpoint)
        print("Loaded model state dict")
    
    model = model.to(device)
    model.eval()
    return model, device

Process and visualize satellite image for feature recognition. In the code below, change "confidence_threshold" to higher number if you want the inference to be very strict in classifying images, or low value if you want to see if you want to recognize more. Note: In my research, I have a following step of verifying the ML inference results with terrain and elevation data (this verification is not in the scope of this Jupyter notebook).

In [ ]:
def process_single_image(image_path: str, model: torch.nn.Module, device: torch.device, 
                        confidence_threshold: float = 0.25) -> Tuple[torch.Tensor, List[str]]:
    """
    Process a single image and return predictions
    """
    # Load and preprocess image
    image = Image.open(image_path).convert('RGB')
    
    # Convert to tensor and normalize
    image_tensor = F.to_tensor(image)
    image_tensor = F.normalize(image_tensor, 
                             mean=[0.485, 0.456, 0.406], 
                             std=[0.229, 0.224, 0.225])
    
    # Get predictions
    with torch.no_grad():
        prediction = model([image_tensor.to(device)])
    
    # Filter by confidence threshold
    mask = prediction[0]['scores'] > confidence_threshold
    boxes = prediction[0]['boxes'][mask]
    labels = [prediction[0]['labels'][i] for i in range(len(prediction[0]['scores'])) 
             if prediction[0]['scores'][i] > confidence_threshold]
    scores = prediction[0]['scores'][mask]
    
    return image, boxes.cpu(), labels, scores.cpu()

In [ ]:
def visualize_image_pair(image_path: str, model: torch.nn.Module, device: torch.device):
    """
    Display original and annotated versions of an image side by side
    """
    # Create figure
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 7))
    
    # Load and display original image
    original_image = Image.open(image_path).convert('RGB')
    ax1.imshow(original_image)
    ax1.set_title('Original Image')
    ax1.axis('off')
    
    # Get predictions and display annotated image
    image, boxes, labels, scores = process_single_image(image_path, model, device)
    
    # Display annotated image
    ax2.imshow(original_image)
    
    # Add bounding boxes and labels
    for box, label, score in zip(boxes, labels, scores):
        color = LABEL_CONFIG[label]
        rgb_color = hex_to_rgb(color)
        
        # Create rectangle patch
        rect = patches.Rectangle(
            (box[0], box[1]),
            box[2] - box[0],
            box[3] - box[1],
            linewidth=2,
            edgecolor=color,
            facecolor='none'
        )
        ax2.add_patch(rect)
        
        # Add label text
        ax2.text(
            box[0], box[1]-5,
            f'{label}: {score:.2f}',
            color=color,
            fontsize=8,
            bbox=dict(facecolor='white', alpha=0.7, edgecolor='none')
        )
    
    ax2.set_title('Annotated Image')
    ax2.axis('off')
    
    plt.tight_layout()
    plt.show()

Chang the code below for the notebook to access the ML model stored in your local machine. Insert the path where you dowloaded and stored the ML model in place of "YOUR LOCAL DIRECTORY FOR ML MODEL"

In [ ]:
# Load the model
#FINAL MODEL
checkpoint_path = r'YOUR LOCAL DIRECTORY FOR ML MODEL/rcnn_model_2025-02-10_19.pth'
model, device = load_checkpoint_model(checkpoint_path)

Using CUDA device: NVIDIA RTX 500 Ada Generation Laptop GPU
Loaded model state dict


Chang the code below for the notebook to access the Google satellite tile images stored in your local machine. Insert the path where you dowloaded and stored the sample images, or the images you downloaded using google earth api, in place of "YOUR LOCAL DIRECTORY FOR IMAGES". 

Note: As explained before, this ML inference works with google map tiles at zoom level 12, 512x512 pixels resolution [at scale 2], and without cloud cover.

In [ ]:
# Visualize image
img_file = [""]*12
# Lakes or Rivers
img_file[0] = "NormColCorr_tile_667_1564_-121.38_39.10.png" # lake & Reservoir-area
img_file[1] = "NormColCorr_tile_670_1546_-121.11_40.31.png"
img_file[2] = "NormColCorr_tile_688_1569_-119.53_38.75.png" # lake & farm
#farm
img_file[3] = "NormColCorr_tile_667_1521_-121.38_41.97.png" 
img_file[4] = "NormColCorr_tile_670_1589_-121.11_37.37.png" # Farm, Suburb & Reservoir-area
img_file[5] = "NormColCorr_tile_671_1587_-121.03_37.51.png"
# Reservoir area
img_file[6] = "NormColCorr_tile_676_1526_-120.59_41.64.png"
img_file[7] = "NormColCorr_tile_708_1585_-117.77_37.65.png"
img_file[8] = "NormColCorr_tile_708_1618_-117.77_35.32.png"
#Suburb
img_file[9] = "NormColCorr_tile_659_1586_-122.08_37.58.png"
img_file[10] = "NormColCorr_tile_666_1583_-121.46_37.79.png" # suburb & farm
img_file[11] = "NormColCorr_tile_667_1572_-121.38_38.55.png"

index_img = 0 # CHANGE THIS INDEX TO TRY MULTIPLE IMAGES.

image_path = r'YOUR LOCAL DIRECTORY FOR IMAGES'
image_path = os.path.join(image_path, img_file[index_img]) # Change index_img
visualize_image_pair(image_path, model, device) 


## ORIGINAL IMAGE AND ANNOTATED IMAGE WILL BE SHOWN BELOW